# Silicium-Aluminum soil concentration: FUNCTIONS
This is a notebook file that contains various functions for the main file.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd 
from scipy.optimize import minimize, basinhopping
from scipy.stats import norm, t, f

% matplotlib notebook

In [ ]:
def log_progress(sequence, every=None, size=None):
    """
    Function that implements a bar to show the progress of the iterations.
    Source: https://github.com/alexanderkuk/log-progress.git
    """
    from ipywidgets import IntProgress, HTML, VBox
    from IPython.display import display

    is_iterator = False
    if size is None:
        try:
            size = len(sequence)
        except TypeError:
            is_iterator = True
    if size is not None:
        if every is None:
            if size <= 200:
                every = 1
            else:
                every = int(size / 200)     # every 0.5%
    else:
        assert every is not None, 'sequence is iterator, set every'

    if is_iterator:
        progress = IntProgress(min=0, max=1, value=1)
        progress.bar_style = 'info'
    else:
        progress = IntProgress(min=0, max=size, value=0)
    label = HTML()
    box = VBox(children=[label, progress])
    display(box)

    index = 0
    try:
        for index, record in enumerate(sequence, 1):
            if index == 1 or index % every == 0:
                if is_iterator:
                    label.value = '{index} / ?'.format(index=index)
                else:
                    progress.value = index
                    label.value = u'{index} / {size}'.format(
                        index=index,
                        size=size
                    )
            yield record
    except:
        progress.bar_style = 'danger'
        raise
    else:
        progress.bar_style = 'success'
        progress.value = index
        label.value = str(index or '?')

In [ ]:
def Model(paras, t):
    """
    Calculates our model values of Si and Al based on a given set of parameters.
    INPUT:
        paras is an ndarray of shape (3n + 3,).
        t is a float or an ndarray of shape (m, ) with the times at which to evaluate the model.
    OUTPUT:
        Si and Al are ndarrays of shape (m, ) with the model values evaluated at the times
            specified by t. If t was a float, these are ndarrays of shape (1, ).
    """
    
    # Determine the number of parameters:
    n = (paras.size - 3)/3
    
    # Check t
    if isinstance(t, float):
        t = np.array([t])
    
    # The linear part:
    t = t - paras[-1]
    Si = paras[-3]*t
    Al = paras[-3]/paras[-2]*t
    
    # The exponential part:
    for i in range(0, n):
        decay = 1 - np.exp(-paras[i]*t)
        Si = Si + paras[n + i]*decay
        Al = Al + paras[n + i]/paras[2*n + i]*decay
    
    return Si, Al

In [ ]:
def gradModel(paras, t):
    """
    Evaluates the gradient of the model.
       INPUT:
           paras is an ndarray of shape (3n + 3,).
           t is a float or an ndarray of shape (m, ) with the times at which to evaluate the model.
       OUTPUT:
           dSi is an ndarray of shape (m, 3n + 3) containing the gradient of the model w.r.t each
               parameter and evaluated at each time in t (if t is a float, m = 1).
           dAl is an ndarray of shape (m, 3n + 3) containing the gradient of the model w.r.t each
               parameter and evaluated at each time in t (if t is a float, m = 1).
    """
    
    # Determine the number of parameters:
    n = (paras.size - 3)/3
    
    # Check t
    if isinstance(t, float):
        t = np.array([t])
        m = 1
    else:
        m = t.size
    
    
    t = t - paras[-1]
    dSi = np.zeros([m, 3*n + 3])
    dAl = np.zeros([m, 3*n + 3])
    
    # Calculate dSi and dAl column by column:
    dSi[:, 3*n] = t
    dSi[:, 3*n + 2] = -paras[-3]
    dAl[:, 3*n] = t/paras[-2]
    dAl[:, 3*n + 1] = -paras[-3]*t/(paras[-2]**2)
    dAl[:, 3*n + 2] = -paras[-3]/paras[-2]
    for i in range(0, n):
        # Updates for dSi:
        dSi[:, i] = paras[n + i]*t*np.exp(-paras[i]*t)
        dSi[:, n + i] = 1 - np.exp(-paras[i]*t)
        dSi[:, 3*n + 2] += -paras[i]*paras[n + i]*np.exp(-paras[i]*t)
        
        # Updates for dAl:
        dAl[:, i] = paras[n + i]/paras[2*n + i]*t*np.exp(-paras[i]*t)
        dAl[:, n + i] = (1 - np.exp(-paras[i]*t))/paras[2*n + i]
        dAl[:, 2*n + i] = -paras[n + i]/(paras[2*n + i]**2)*(1 - np.exp(-paras[i]*t))
        dAl[:, 3*n + 2] += -paras[i]*paras[n + i]/paras[2*n + i]*np.exp(-paras[i]*t)

    #
    return dSi, dAl

In [ ]:
def Cost(paras, SiBar, AlBar, time):
    """
    Evaluates the cost function.
       INPUT:
           paras is an ndarray of shape (3n + 3,).
           SiBar is an ndarray containing the Si measurements.
           AlBar is an ndarray containing the Al measurements.
           time is an ndarray containing the measurementtimes.
       OUTPUT:
           The output is a float, the evaluation of the cost function for the given set of parameters.
    """
    
    # Calcuate model prediction:
    Si, Al = Model(paras, time)
    
    # Return the cost:
    return np.sum((Si - SiBar)**2) + np.sum((Al - AlBar)**2)

In [ ]:
def gradCost(paras, SiBar, AlBar, time):
    """
    Evaluates the gradient of the cost function.
       INPUT:
           paras is an ndarray of shape (3n + 3,).
           SiBar is an ndarray containing the Si measurements.
           AlBar is an ndarray containing the Al measurements.
           time is an ndarray containing the measurementtimes.
       OUTPUT:
           grad is an ndarray of shape (3n + 3,) containing the gradient of the cost function w.r.t
               each of the parameter.
    """
    
    Si, Al = Model(paras, time)
    dSi, dAl = gradModel(paras, time)
    grad = 2*(np.matmul((Si - SiBar), dSi) + np.matmul((Al - AlBar), dAl))
    
    return grad

In [ ]:
def hesstildeCost(paras, SiBar, AlBar, time):
    """
    Calculates an approximation to the Hessian of the cost function.
       INPUT:
           paras is an ndarray of shape (3n + 3,).
           SiBar is an ndarray containing the Si measurements.
           AlBar is an ndarray containing the Al measurements.
           time is an ndarray containing the measurementtimes.
       OUTPUT:
           Returns an ndarray of shape (3n + 3, 3n + 3).
    """
    def hessMatvec(x):
        h = np.sqrt(1.1e-16)
        h1 = gradCost(paras - h*x, SiBar, AlBar, time)
        h2 = gradCost(paras + h*x, SiBar, AlBar, time)
        
        return (h2 - h1)/(2*h)
    
    n = paras.size
    H = np.zeros((n, n))
    for i in range(0, n):
        e = np.zeros(n)
        e[i] = 1
        H[i, :] = hessMatvec(e)
    
    l, U = np.linalg.eig(H)
    return np.dot(abs(l)*U, np.linalg.inv(U))

In [ ]:
def StochasticNewtonMCMC(x0, SiBar, AlBar, time, samples = 1000):
    """
    Implementation of the Stochastic Newton MCMC method: 
            Martin, James, et al. "A stochastic Newton MCMC method for large-scale statistical inverse problems
            with application to seismic inversion." SIAM Journal on Scientific Computing 34.3 (2012): A1460-A1487.
       INPUT:
           paras is an ndarray of shape (3n + 3,).
           SiBar is an ndarray containing the Si measurements.
           AlBar is an ndarray containing the Al measurements.
           time is an ndarray containing the measurementtimes.
       OUTPUT:
           results is a dictonarry containing the results of the algorithm."""
    k = x0.size
    n = (k - 3)/3
    bnds = []
    for i in range(0, 2*n):
        bnds.append((0., None))
    for i in range(2*n, 3*n):
        bnds.append((1e-16, None))
    bnds.append((0, None))
    bnds.append((1e-16, None))
    bnds.append((None, None))
    bnds = tuple(bnds)
    
    flag_hess = 0
    flag_naninf = 0
    
    # Make sure we start in a local minimum:
    results = minimize(Cost, x0, bounds = bnds, args = (SiBar, AlBar, time))
    
    #
    x = np.zeros((samples + 1, k))
    x[0, :] = results.x
    
    #
    f = np.zeros(samples + 1)
    f[0] = Cost(x[0, :], SiBar, AlBar, time)
    
    #
    try:
        Hx = hesstildeCost(x[0, :], SiBar, AlBar, time)
        Lx = np.linalg.cholesky(Hx)
    except np.linalg.LinAlgError:
        Lx = np.identity(k)
        flag_hess += 1
    
    # Optimal point:
    xopt = x[0, :]
    fopt = f[0]
    
    # Monte Carlo:
    for i in log_progress(range(samples), every = 1):
        for j in range(5):
            # Draw a sample and optimize
            y = x[i, :] + np.linalg.solve(Lx, np.random.normal(size = k))
            results = minimize(Cost, y, bounds = bnds, args = (SiBar, AlBar, time))
            
            try:
                Hy = hesstildeCost(results.x, SiBar, AlBar, time)
                Ly = np.linalg.cholesky(Hy)
            except np.linalg.LinAlgError:
                Ly = np.identity(k)
                if j == 5:
                    flag_hess += 1
            
            if np.isnan(results.fun) or np.isinf(results.fun):
                if j == 5:
                    flag_naninf += 1
                continue
            else:
                break
        
        # Metropolis-Hastings:
        alpha = min([0.9, f[i]/results.fun])
        if np.random.rand() < alpha:
            x[i + 1, :] = results.x
            f[i + 1] = results.fun
            Lx = Ly
            if f[i + 1] < fopt:
                xopt = x[i + 1, :]
                fopt = f[i + 1]
        else:
            x[i + 1, :] = x[i, :]
            f[i + 1] = f[i]
    
    results = {'x':xopt, 'fun':fopt, 'samples':x, 'samplecost':f, 'flag_hess':flag_hess, 'flag_naninf':flag_naninf}
    return results

In [ ]:
def StochasticNewtonMCMC_mute(x0, SiBar, AlBar, time, samples = 1000):
    """
    Identical to StochasticNewtonMCMC, but without the waitbar.
    """
    k = x0.size
    n = (k - 3)/3
    bnds = []
    for i in range(0, 2*n):
        bnds.append((0., None))
    for i in range(2*n, 3*n):
        bnds.append((1e-16, None))
    bnds.append((0, None))
    bnds.append((1e-16, None))
    bnds.append((None, None))
    bnds = tuple(bnds)
    
    flag_hess = 0
    flag_naninf = 0
    
    # Make sure we start in a local minimum:
    results = minimize(Cost, x0, bounds = bnds, args = (SiBar, AlBar, time))
    
    #
    x = np.zeros((samples + 1, k))
    x[0, :] = results.x
    
    #
    f = np.zeros(samples + 1)
    f[0] = Cost(x[0, :], SiBar, AlBar, time)
    
    #
    try:
        Hx = hesstildeCost(x[0, :], SiBar, AlBar, time)
        Lx = np.linalg.cholesky(Hx)
    except np.linalg.LinAlgError:
        Lx = np.identity(k)
        flag_hess += 1
    
    # Optimal point:
    xopt = x[0, :]
    fopt = f[0]
    
    # Monte Carlo:
    for i in range(samples):
        for j in range(5):
            # Draw a sample and optimize
            y = x[i, :] + np.linalg.solve(Lx, np.random.normal(size = k))
            results = minimize(Cost, y, bounds = bnds, args = (SiBar, AlBar, time))
            
            try:
                Hy = hesstildeCost(results.x, SiBar, AlBar, time)
                Ly = np.linalg.cholesky(Hy)
            except np.linalg.LinAlgError:
                Ly = np.identity(k)
                if j == 5:
                    flag_hess += 1
            
            if np.isnan(results.fun) or np.isinf(results.fun):
                if j == 5:
                    flag_naninf += 1
                continue
            else:
                break
        
        # Metropolis-Hastings:
        alpha = min([0.9, f[i]/results.fun])
        if np.random.rand() < alpha:
            x[i + 1, :] = results.x
            f[i + 1] = results.fun
            Lx = Ly
            if f[i + 1] < fopt:
                xopt = x[i + 1, :]
                fopt = f[i + 1]
        else:
            x[i + 1, :] = x[i, :]
            f[i + 1] = f[i]
    
    results = {'x':xopt, 'fun':fopt, 'samples':x, 'samplecost':f, 'flag_hess':flag_hess, 'flag_naninf':flag_naninf}
    return results

In [ ]:
"""
If x is a set of parameters, then the following functions reorder the parameters based on
either the largest value of k, AlkExSi or SiAl.
"""

def Sort_By_k(x):
    n = (len(x) - 3)/3
    y = np.reshape(x[:3*n], (3, n))
    y = y[:, y[0, :].argsort()]
    return np.concatenate([np.reshape(y, 3*n), x[-3:]])

def Sort_By_AlkExSi(x):
    n = (len(x) - 3)/3
    y = np.reshape(x[:3*n], (3, n))
    y = y[:, y[1, :].argsort()]
    return np.concatenate([np.reshape(y, 3*n), x[-3:]])

def Sort_By_SiAl(x):
    n = (len(x) - 3)/3
    y = np.reshape(x[:3*n], (3, n))
    y = y[:, y[2, :].argsort()]
    return np.concatenate([np.reshape(y, 3*n), x[-3:]])